<a href="https://colab.research.google.com/github/ansonkwokth/PlackettLuceModel/blob/main/example_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install package from github

In [1]:
# # Clone the repository
# !git clone https://github.com/ansonkwokth/PlackettLuceModel.git

# # Navigate to the project directory
# %cd /content/PlackettLuceModel

# # Install dependencies
# !pip install .

Cloning into 'PlackettLuceModel'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 118 (delta 51), reused 30 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (118/118), 37.13 KiB | 2.06 MiB/s, done.
Resolving deltas: 100% (51/51), done.
/content/PlackettLuceModel
Processing /content/PlackettLuceModel
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for plackett_luce: filename=plackett_luce-0.1.0-py3-none-any.whl size=2130 sha256=25db44a45df744ffe5a946acb14a6229af263b8896ddf579e3a114581258fb0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-qf7y_ym5/wheels/a8/f1/2b/b543467c5381cc96021439e6b82036714dcdc447163a28517a
Successfully built plackett_luce


### Create the example data set

In [2]:
# !python ./scripts/generate_example_data.py

/content/PlackettLuceModel


## Example usage

In [3]:
%cd /content/PlackettLuceModel
from plackett_luce.utils import DataLoader
from plackett_luce.model import PlackettLuceModel
from plackett_luce.utils import EarlyStopper

import pandas as pd
import numpy as np
import torch
from torch import nn

torch.manual_seed(0);

/content/PlackettLuceModel


In [4]:
# Reading csv
df = pd.read_csv("./data/example_data/example_data.csv")

In [5]:
# Transforming df into model data format
X, rankings, mask = DataLoader().transform(df)

### Training model

In [6]:
# for simplicity, just pick the first 1000 to train
n_train = 1000
X_train = X[:n_train]
rankings_train = rankings[:n_train]
mask_train = mask[:n_train]

In [9]:
# Custom neural network model for flexible scoring
class LinearReg(nn.Module):
    def __init__(self, input_dim):
        super(LinearReg, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 1)
        )

    def forward(self, x):
        return self.network(x)



# Custom neural network model for flexible scoring
class NaiveNN(nn.Module):
    def __init__(self, input_dim):
        super(NaiveNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1)  # 1D output for scoring
        )

    def forward(self, x):
        return self.network(x)



# Custom neural network model for flexible scoring
class LessNaiveNN(nn.Module):
    def __init__(self, input_dim):
        super(LessNaiveNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1)  # 1D output for scoring
        )

    def forward(self, x):
        return self.network(x)

In [16]:
# Initialize the model
num_features = X_train.shape[-1]
# custom_nn = LinearReg(input_dim=num_features)
# custom_nn = NaiveNN(input_dim=num_features)
custom_nn = LessNaiveNN(input_dim=num_features)

# Custom early stopper
custom_early_stopper = EarlyStopper(patience=5, min_delta=0.01)
model = PlackettLuceModel(score_model=custom_nn, early_stopper=custom_early_stopper)
print(f"Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

# Training
print("Training the model...")

model.fit(X_train, rankings_train, lr=0.01, epochs=500, top_k=3, item_mask=mask_train)



Trainable params: 433
Training the model...
Epoch 10/500, Negative Log-Likelihood: 7.3644
Epoch 20/500, Negative Log-Likelihood: 6.4733
Epoch 30/500, Negative Log-Likelihood: 5.3179
Epoch 40/500, Negative Log-Likelihood: 4.5474
Epoch 50/500, Negative Log-Likelihood: 4.1401
Epoch 60/500, Negative Log-Likelihood: 3.9446
Epoch 70/500, Negative Log-Likelihood: 3.8218
Epoch 80/500, Negative Log-Likelihood: 3.7418
Early stopping at epoch 82 with NLL 3.7301


### Testing model

In [20]:
# for simplicity, just pick the last 10 to train
n_test = -10
X_test = X[n_test:]
rankings_test = rankings[n_test:]
mask_test = mask[n_test:]

In [21]:
# prediction
pred_test = model.predict(X_test)
pred_test = torch.tensor(pred_test)
# replace the dumpy entry by -99
pred_test[mask_test == 0] = -99

In [22]:

for i in range(10):
    print("instance", i+1)
    print("Pred first 5", pred_test[i][:5].tolist())
    print("True first 5", rankings_test[i][:5].tolist())
    print()

instance 1
Pred first 5 [6, 7, 9, 8, 1]
True first 5 [6, 9, 3, 7, 8]

instance 2
Pred first 5 [0, 4, 3, 1, 6]
True first 5 [0, 4, 1, 3, 6]

instance 3
Pred first 5 [1, 4, 2, 10, 7]
True first 5 [1, 3, 4, 0, 10]

instance 4
Pred first 5 [11, 1, 12, 0, 10]
True first 5 [11, 12, 0, 5, 2]

instance 5
Pred first 5 [6, 5, 3, 1, 2]
True first 5 [3, 6, 2, 5, 1]

instance 6
Pred first 5 [6, 4, 8, 9, 0]
True first 5 [6, 8, 9, 4, 0]

instance 7
Pred first 5 [8, 13, 12, 2, 9]
True first 5 [13, 12, 8, 9, 2]

instance 8
Pred first 5 [9, 6, 7, 2, 3]
True first 5 [6, 2, 7, 4, 3]

instance 9
Pred first 5 [6, 8, 2, 4, 1]
True first 5 [8, 6, 4, 0, 2]

instance 10
Pred first 5 [1, 5, 0, 8, 3]
True first 5 [3, 1, 5, 8, 0]

